# ファインチューニングの実験 

## 事前学習済みモデル 
rinna社の日本語GPT-2 [1, 2] (**japanese-gpt2-medium** [3]) を用いる.  
3.36億パラメータ, 24層のモデルで, 学習にはメモリ32GBのV100を8台で45日を要したとある.

## データセット 
**おーぷん2ちゃんねる対話コーパス** [4, 5] を用いる (こちら [6] で知った).  
およそ約815万件の対話データを収録. 無償で商用利用も可.  
このうち, `newsplus.tsv` の前から1万件のみを用いる. 


### 準備 


In [ ]:
# GPUの確認
!nvidia-smi


In [ ]:
# パッケージのインストール
!git clone https://github.com/huggingface/transformers -b v4.23.1
!pip install transformers==4.23.1
!pip install evaluate==0.3.0
!pip install sentencepiece==0.1.97


### Googleドライブの作業フォルダ「work」の作成 


In [ ]:
from google.colab import drive

drive.mount("/content/drive")
!mkdir -p "/content/drive/MyDrive/work"
%cd "/content/drive/MyDrive/work"


### 前処理 

あらかじめ `dir_name` 以下に, コーパスのデータを置いておく. 


In [ ]:
import os
import re

# このディレクトリにデータを置いておく
dir_name = "data/open2ch-dialogue-corpus/corpus"

def preprocess(input_file_name, output_file_name, max_n_lines=None, output_mode="w"):
  """
  前処理では, 
   ・ 改行記号「 __BR__ 」が1つ〜3つ連続の場合, 半角空白1つへ置換
   ・ タブ記号を半角空白1つへ置換

  前処理した結果をファイルへ書込み. output_modeに従う
   ・ 既定値: "w" (上書きモード)
   ・ "a" (追記モード)
  """

  input_file_path = os.path.join(dir_name, input_file_name)
  output_file_path = os.path.join(dir_name, output_file_name)

  texts = []
  print_steps = 10000  # 何件ごとに進捗を表示するか
  with open(input_file_path) as f:
    for i, l in enumerate(f):

      # 最大行数が指定されていて, かつ, それに達したら終了
      if max_n_lines is not None and max_n_lines <= i:
        break

      # 改行記号は「 __BR__ 」へ置換されている. これを半角空白1つへ置換する
      l = re.sub(" __BR__ __BR__ __BR__ ", " ", l)  # 3つ連続の場合
      l = re.sub(" __BR__ __BR__ ", " ", l)  # 2つ連続の場合
      l = re.sub(" __BR__ ", " ", l)  # 1つの場合

      # レスアンカー「>>」はタブ記号へ置換されている. これを半角空白1つへ置換する
      l = re.sub("\t", " ", l)

      texts.append(l)

      # 何件かごとに進捗を表示
      if (i+1) % print_steps == 0:
        print(f"{i+1:9,} 件 Done!")

  # 書込み
  with open(output_file_path, output_mode) as f:
    f.writelines(texts)

  return output_file_path


In [ ]:
%%time

"""
コーパスには3つのファイルがある
 ・ livejupiter.tsv (5,948,218 行)
 ・ news4vip.tsv (1,983,626 行)
 ・ newsplus.tsv (217,296 行)
"""
DATA_NAME = "newsplus"  # 学習データ名
DATA_SIZE = 1  # 学習データに何「万」件用いるか

input_file_name = f"{DATA_NAME}.tsv"
output_file_name = f"preprocessed_{DATA_NAME}.txt"

output_file_path = preprocess(
    input_file_name, output_file_name, max_n_lines=DATA_SIZE*10000, output_mode="w")

print(f"\n前処理済みテキストファイルのパスは, \n\t{output_file_path}\nです. \n")


   10,000 件 Done!

前処理済みテキストファイルのパスは, 
	data/open2ch-dialogue-corpus/corpus/preprocessed_newsplus.txt
です. 

CPU times: user 100 ms, sys: 13.3 ms, total: 113 ms
Wall time: 548 ms


### 学習 

* `--model_name_or_path` に, 事前学習済みモデルのパスを指定. 
* `--train_file` と `--validation_file` の両方に, 前処理済みテキストファイルのパスを指定. 

モデルが `rinna/japanese-gpt2-medium` の場合, 
* `--per_device_train_batch_size` に `1` を指定しないと, メモリ不足で学習できなかった. 
* `--per_device_eval_batch_size` は `8` が限界だった. 


In [ ]:
%%time

!python /content/transformers/examples/pytorch/language-modeling/run_clm.py \
    --model_name_or_path=rinna/japanese-gpt2-medium \
    --train_file=data/open2ch-dialogue-corpus/corpus/preprocessed_newsplus.txt \
    --validation_file=data/open2ch-dialogue-corpus/corpus/preprocessed_newsplus.txt \
    --do_train \
    --do_eval \
    --num_train_epochs=2 \
    --save_steps=100 \
    --save_total_limit=3 \
    --per_device_train_batch_size=1 \
    --per_device_eval_batch_size=8 \
    --output_dir=output/ \
    --overwrite_output_dir=true


### ファインチューニング後の推論結果 

In [ ]:
from transformers import T5Tokenizer, AutoModelForCausalLM


INPUT_TEXT = "ww"  # 入力するテキスト


def print_result(result):
  """
  結果を整理して表示
  """

  for i, l in enumerate(result):
    print(f"\n◆ {i+1}番めの結果: \n{l}\n")


In [ ]:
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
model = AutoModelForCausalLM.from_pretrained("output/")


In [ ]:
%%time

output = model.generate(
    tokenizer.encode(INPUT_TEXT, return_tensors="pt"), 
    do_sample=True, max_length=100, num_return_sequences=8
)

# 結果を整理して表示
print_result(tokenizer.batch_decode(output, skip_special_tokens=True))


### ファインチューニング前の推論結果 


In [ ]:
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-medium")


In [ ]:
%%time

output = model.generate(
    tokenizer.encode(INPUT_TEXT, return_tensors="pt"), 
    do_sample=True, max_length=100, num_return_sequences=8
)

# 結果を整理して表示
print_result(tokenizer.batch_decode(output, skip_special_tokens=True))


### 参考 

1. rinnakk, "japanese-pretrained-models", GitHub.  
  https://github.com/rinnakk/japanese-pretrained-models 

2. 趙 天雨, 沢田 慶, "日本語自然言語処理における事前学習モデルの公開", 人工知能学会研究会資料 言語・音声理解と対話処理研究会, vol.93, pp.169-170, 2021. 

3. rinna, "japanese-gpt2-medium", Hugging Face.  
  https://huggingface.co/rinna/japanese-gpt2-medium 

4. 1never, "おーぷん2ちゃんねる対話コーパス", GitHub.  
  https://github.com/1never/open2ch-dialogue-corpus 

5. 稲葉 通将, "おーぷん2ちゃんねる対話コーパスを用いた用例ベース対話システム", 第87回言語・音声理解と対話処理研究会(第10回対話システムシンポジウム), 人工知能学会研究会資料 SIG-SLUD-B902-33, pp.129-132, 2019. 

6. 水上 雅博, 吉野 幸一郎, 中野 幹生, 赤間 怜奈, 駒谷 和範, 吉川 禎洋, 林部 祐太, 児玉 貴志, "日本語対話コーパス".  
  https://masahiro-mi.github.io/dialogue_corpus.html 

